## Baseline with natasha

In [1]:
%load_ext autotime

In [35]:
import pandas as pd
import numpy as np
import re

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

from natasha import NamesExtractor

time: 8 ms


In [3]:
train = pd.read_csv('train.csv', encoding='utf-8')
test = pd.read_csv('test.csv', encoding='utf-8')
sample_submission = pd.read_csv('sample_submission.csv')
train.head()

,Word,Label
0,Аалтонен,1
1,Аар,0
2,Аарон,0
3,ААРОН,0
4,Аарона,0


time: 618 ms


In [65]:
vowels = ['а',  'я', 'ё', 'у','е', 'о', 'э', 'ю', 'и', 'ы', 'Ё', 'У', 'Е', 'Ы','А', 'О', 'Э', 'Ю', 'И', 'Я']

extractor = NamesExtractor()

def has_name(text):
    matches = extractor(text).as_json
    if matches !=[]:
        for match in matches:
            if match['fact']:
                if match['fact']['last']:
                    return 1
    return 0

def prepare_features(df):
    df['Lenght'] = df['Word'].apply(lambda x: len(x))
    df['Vowels'] = df['Word'].apply(lambda x: sum(1 if l in vowels else 0 for l in x))
    df['Consonants'] = df['Lenght'] - df['Vowels']
    df['Not_null_Consonants'] = df['Consonants'].apply(lambda x: 0.001 if x==0 else x)
    df['Vow/Conson'] = df['Vowels'] / df['Not_null_Consonants']
    df = df.drop(columns=['Not_null_Consonants'])
    df['is_lower'] = df['Word'].apply(lambda x: 1 if x[0] == x[0].lower() else 0)
    df['has_name'] = df['Word'].apply(lambda word: has_name(word))
    return df

time: 38.1 ms


In [66]:
train_all = prepare_features(train)
test_all = prepare_features(test)

x_train = train_all.drop(columns=['Word', 'Label'])
y_train = train_all['Label']
x_test = test_all.drop(columns=['Word'])

time: 3min 17s


In [25]:
train_all.head()

,Word,Label,Lenght,Vowels,Consonants,Vow/Conson,is_lower
0,Аалтонен,1,8,4,4,1.0,0
1,Аар,0,3,2,1,2.0,0
2,Аарон,0,5,3,2,1.5,0
3,ААРОН,0,5,3,2,1.5,0
4,Аарона,0,6,4,2,2.0,0


time: 12 ms


Crossval score on train

In [34]:
np.mean(cross_val_score(LogisticRegression(), x_train, y_train, cv=5))

0.9001657133408149

time: 1.21 s


In [67]:
np.mean(cross_val_score(LogisticRegression(), x_train, y_train, cv=5))

0.9237534784001961

time: 1.59 s


Create baseline submit

In [38]:
logreg = LogisticRegression()
logreg.fit(x_train, y_train)
y_predict = logreg.predict_proba(x_test)

submit = pd.DataFrame()
submit['Id'] = test.index
submit['Prediction'] = y_predict[:,1]
submit.to_csv('benchmark.csv', index = False)
submit[:5]

,Id,Prediction
0,0,0.126227
1,1,0.171074
2,2,0.121990
3,3,0.121990
4,4,0.530969


time: 1.06 s
